In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("test")
sc = SparkContext(conf=conf)
                              
sqlCtx = SQLContext(sc)
df = sqlCtx.read.load('file:///home/fran/Descargas/p2/Splitted/task3_dataset_cortado.csv', 
                     format='com.databricks.spark.csv', header='true', inferSchema='true')
df2 = sqlCtx.read.load('file:///home/fran/Descargas/p2/Splitted/task3_dataset_cortado_solo_ataques.csv', 
                     format='com.databricks.spark.csv', header='true', inferSchema='true')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=test, master=local[*]) created by __init__ at <ipython-input-1-6e82a18047ae>:5 

In [4]:

df2.columns

['_c0',
 'GyroscopeStat_x_MEAN',
 'GyroscopeStat_z_MEAN',
 'GyroscopeStat_COV_z_x',
 'GyroscopeStat_COV_z_y',
 'MagneticField_x_MEAN',
 'MagneticField_z_MEAN',
 'MagneticField_COV_z_x',
 'MagneticField_COV_z_y',
 'Pressure_MEAN',
 'LinearAcceleration_COV_z_x',
 'LinearAcceleration_COV_z_y',
 'LinearAcceleration_x_MEAN',
 'LinearAcceleration_z_MEAN',
 'attack']

In [5]:
df.printSchema()


root
 |-- _c0: integer (nullable = true)
 |-- GyroscopeStat_x_MEAN: double (nullable = true)
 |-- GyroscopeStat_z_MEAN: double (nullable = true)
 |-- GyroscopeStat_COV_z_x: double (nullable = true)
 |-- GyroscopeStat_COV_z_y: double (nullable = true)
 |-- MagneticField_x_MEAN: double (nullable = true)
 |-- MagneticField_z_MEAN: double (nullable = true)
 |-- MagneticField_COV_z_x: double (nullable = true)
 |-- MagneticField_COV_z_y: double (nullable = true)
 |-- Pressure_MEAN: double (nullable = true)
 |-- LinearAcceleration_COV_z_x: double (nullable = true)
 |-- LinearAcceleration_COV_z_y: double (nullable = true)
 |-- LinearAcceleration_x_MEAN: double (nullable = true)
 |-- LinearAcceleration_z_MEAN: double (nullable = true)
 |-- attack: integer (nullable = true)



In [8]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
_c0,38,24498.5,11.113055385446435,24480,24517
GyroscopeStat_x_MEAN,38,-0.002919810384414524,0.042653074223189466,-0.12934446667563454,0.192444842939196
GyroscopeStat_z_MEAN,38,-0.020669563899161792,0.06558910434958976,-0.4005862796940704,0.01580415402964103
GyroscopeStat_COV_z_x,38,-0.004745679981555173,0.03900978043943563,-0.2293913810431748,0.05479838414699555
GyroscopeStat_COV_z_y,38,0.0027225391041282225,0.04253071796921174,-0.09230434233181373,0.23981542270362485
MagneticField_x_MEAN,38,-828.5815522391483,5068.768853339744,-31252.0,42.310158730158726
MagneticField_z_MEAN,38,-20.569553774133187,16.276224724990833,-61.70115789473685,7.576736842105262
MagneticField_COV_z_x,38,3.6145847662909802,10.42308323200004,-45.43701104356637,29.967837894736828
MagneticField_COV_z_y,38,4.417610595486042,6.088725194969065,-7.984286527514232,22.258744782168197


In [42]:
featureColumns= ['GyroscopeStat_x_MEAN', 'Pressure_MEAN', 'LinearAcceleration_z_MEAN', 'GyroscopeStat_z_MEAN']
dfWN=df.na.drop()
dfWN.count(), len(dfWN.columns)

(1586823, 13)

In [43]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler= VectorAssembler (inputCols= featureColumns, outputCol= "features")
assembled = assembler.transform(df)
assembled2 = assembler.transform(df2)

(trainingData, testData) = assembled.randomSplit([0.8,0.2], seed= 13234)
(trainingData2, testData2) = assembled2.randomSplit([0.8,0.2], seed= 13234)

trainingData = trainingData.union(trainingData2)
testData = testData.union(testData2)
trainingData2.count(), testData2.count(), 




IllegalArgumentException: 'Field "GyroscopeStat_x_MEAN" does not exist.\nAvailable fields: rowID, hpwren_timestamp, air_pressure, air_temp, avg_wind_direction, avg_wind_speed, max_wind_direction, max_wind_speed, min_wind_direction, min_wind_speed, rain_accumulation, rain_duration, relative_humidity'

In [25]:
from pyspark.ml.classification import DecisionTreeClassifier
dt= DecisionTreeClassifier (labelCol="attack", featuresCol="features",
                           maxDepth=5, minInstancesPerNode=20,
                           impurity="gini")

In [26]:
from pyspark.ml import Pipeline

pipeline= Pipeline(stages=[dt])
model = pipeline.fit (trainingData)

In [27]:
predictions= model.transform(testData)

In [28]:
predictions.select("prediction", "attack").show(10)

+----------+------+
|prediction|attack|
+----------+------+
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
+----------+------+
only showing top 10 rows



In [30]:
predictions.select("prediction", "attack").write.save(path="file:///home/fran/Descargas/p2/predictions",format="com.databricks.spark.csv", header="true")

In [31]:

predictions=sqlCtx.read.load('file:///home/fran/Descargas/p2/predictions', format='com.databricks.spark.csv', header='true',inferSchema='true')

In [32]:
predictions.show(10)

+----------+------+
|prediction|attack|
+----------+------+
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
+----------+------+
only showing top 10 rows



In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator( labelCol="attack", predictionCol="prediction", metricName="accuracy")
acuracy= evaluator.evaluate(predictions)
print("Acuraccy = %g" % (acuracy))


Acuraccy = 0.998367


In [39]:
from pyspark.mllib.evaluation import MulticlassMetrics
#predictions = (predictions.withColumn("attack", predictions["attack"].cast("double")))
metrics= MulticlassMetrics(predictions.rdd.map(tuple))

In [40]:
metrics.confusionMatrix().toArray().transpose()

array([[4892.,    8.],
       [   0.,    0.]])